In [17]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [18]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = "./logs/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
# Define the ticker symbol for the stock you want to fetch data for
ticker_symbol = "AAPL"

# Fetch the stock data using yfinance
stock_data = yf.download(ticker_symbol, start="2018-01-01", end="2021-12-31")


[*********************100%%**********************]  1 of 1 completed


In [39]:
Date = stock_data.index
X = stock_data[['Open', 'High', 'Low', 'Volume']]
# Save the date as a feature but make it a float
X['Date'] = (stock_data.index - stock_data.index.min()) / np.timedelta64(1, 'D')
y = stock_data['Close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

/tmp/ipykernel_36559/144902766.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Date'] = (stock_data.index - stock_data.index.min()) / np.timedelta64(1, 'D')


In [40]:
RNN_model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]), # De none in inputlayer betekent dat er geen vaste groote is
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])
RNN_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


/opt/conda/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
RNN_model.fit(X_train, y_train, epochs=10, callbacks=[tensorboard_callback])
test_loss, test_acc = RNN_model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

Epoch 1/10


26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0000e+00 - loss: 5827.9644
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 5282.0449
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 5161.8662
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 5022.6821
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 5179.8477
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 4823.9019
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 4625.3291
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0000e+00 - loss: 4532.1514
Epoch 9/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 4607.6890
Epoch 10/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0000e+00 - loss: 4604.6616
7/7 - 1s - 85ms/step - accuracy: 0.0000e+00 - loss: 19102.5352

Test accuracy: 0.0


In [52]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Load the dataset
df = pd.read_csv('AAPL.csv')  # replace with your file path

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

# Create a data structure with 60 time-steps and 1 output
X = []
y = []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=1, batch_size=1)
test_loss, test_acc = RNN_model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

/opt/conda/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4816/4816 ━━━━━━━━━━━━━━━━━━━━ 304s 62ms/step - loss: 0.0013


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step
Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=True
  • mask=None